In [44]:
import pandas as pd
import numpy as np
import re
import random

In [45]:
df = pd.read_excel (r'Trustworthy_Master_Spreadsheet_Summer_2022.xlsx')
df = df[df["Trustworthy Response"].notnull()]

In [46]:
df

,1,Unnamed: 1,ResponseID,Trustworthy Response,Course,Semester,Semester_Course,PRE/POST,Order,First 10%,...,80-90%,90-100%,Expected,Consistent Results,Uncertainty,Good Methods,Ethics,Peer Review,Statistics,Unnamed: 26
0,2,0,151,An experimental result is acceptable or trustw...,1112,S2019,S2019_1112,PRE,1000,Yes,...,No,No,1,0,0,0,0,0,0,False
1,3,12,R_1hQmOj7GTjnQ38I,After the experiment has been tested by multip...,2213,S2021,S2021_2213,POST,2012,No,...,No,No,0,1,0,0,0,0,0,False
2,4,251,369,I can determine this by calculating the percen...,1112,S2019,S2019_1112,PRE,1251,No,...,No,No,1,0,0,0,0,0,0,False
3,5,253,R_2fDP8D8amSq1e4m,if i can repeat the experiment and get similar...,1110,S2022,S2022_1110,POST,5253,No,...,No,No,0,1,0,0,0,0,0,False
4,6,90,R_1GJNwPP1rE2g8Lf,The experiment is repeated and reviewed by oth...,2213,S2021,S2021_2213,POST,2090,No,...,Yes,No,0,1,0,0,0,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,1706,194,R_1AAYOLmnsmutBFT,The data is acceptable and trustworthy when a ...,1110,S2022,S2022_1110,PRE,4194,No,...,No,No,0,1,0,0,0,0,0,False
1705,1707,85,352,The best way to find if an experimental result...,1112,S2019,S2019_1112,PRE,1085,No,...,No,No,0,1,0,0,0,0,0,False
1706,1708,243,R_2wmpIBomQUhlgQJ,"If they have a sound method, analysis, and acc...",1110,S2022,S2022_1110,POST,5243,No,...,No,No,0,0,1,1,0,0,0,False
1707,1709,180,R_2ByBPaKRXTCLYFy,If the result is repeatable over many trials. ...,1112,S2021,S2021_1112,POST,3180,No,...,No,No,0,1,0,0,0,0,1,False


In [47]:
def set_random_seed(seed=0):
  np.random.seed(seed)
  random.seed(seed)
  #tf.random.set_seed(seed)

In [17]:
from sklearn import model_selection, naive_bayes, svm, linear_model, ensemble, neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer

In [18]:
def cohens_kappa(tp,fp,fn,tn):
    num = 2* (tp*tn - fn*fp)
    denom = (tp+fp)*(fp+tn) + (tp+fn)*(fn+tn)
    return num/denom

In [32]:
#word pre-processing here
def preprocess_text(para):
    # Remove punctuations and numbers
    para = re.sub('[^a-zA-Z]', ' ', para)
    # Single character removal
    para = re.sub(r"\s+[a-zA-Z]\s+", ' ', para)
    # Removing multiple spaces
    para = re.sub(r'\s+', ' ', para)
    
    tokens = para.split()
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    
    return tokens

Xt = []
for response in df["Trustworthy Response"].tolist():
    Xt.append(preprocess_text(response))
    
y = np.array(df["Consistent Results"].tolist())

In [33]:
def make_split(s):
    set_random_seed(seed=s)
    Train_X, Test_X, Train_y, Test_y = model_selection.train_test_split(Xt,y,test_size=0.1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(Train_X)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(Train_X, mode="binary")
    # encode training data set
    Xtest = tokenizer.texts_to_matrix(Test_X, mode="binary")
    return tokenizer, Xtrain, Xtest, Train_y, Test_y

In [34]:
def naivebayes1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    # fit the training dataset on the NB classifier
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_NB = Naive.predict(Xtest)
    # Use accuracy_score function to get the accuracy
    #print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_y)*100)
    #print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_y)*100)
    #print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_y)*100)
    # Confusion Matrix
    #print("Naive Bayes Confusion Matrix -> ",confusion_matrix(predictions_NB, Test_y))
    tn, fp, fn, tp = confusion_matrix(predictions_NB, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_NB, Test_y),precision_score(predictions_NB, Test_y),recall_score(predictions_NB, Test_y),k

In [35]:
def svm1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_SVM, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_SVM, Test_y),precision_score(predictions_SVM, Test_y),recall_score(predictions_SVM, Test_y),k

In [36]:
def logreg1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    Log = linear_model.LogisticRegression(random_state = s, max_iter = 10000)
    Log.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_Log = Log.predict(Xtest)
    # Use accuracy_score function to get the accuracy
    #print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_Log, Test_y)*100)
    # Confusion Matrix
    #print("Logistic Regression Confusion Matrix -> ",confusion_matrix(predictions_Log, Test_y))
    tn, fp, fn, tp = confusion_matrix(predictions_Log, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_Log, Test_y),precision_score(predictions_Log, Test_y),recall_score(predictions_Log, Test_y),k

In [37]:
def randFor1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    RF = ensemble.RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =s)
    RF.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_RF = RF.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_RF, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_RF, Test_y),precision_score(predictions_RF, Test_y),recall_score(predictions_RF, Test_y),k

In [38]:
def KNN1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    kNN = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
    kNN.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_kNN = kNN.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_kNN, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_kNN, Test_y),precision_score(predictions_kNN, Test_y),recall_score(predictions_kNN, Test_y),k

In [39]:
acc = []
prec = []
rec = []
kap = []
for seed in range(40):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = svm1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("SVM")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

SVM
Accuracy:
0.8952941176470588
0.027970572771691148
Precision:
0.9070165818581216
0.024321384429304236
Recall:
0.9160505887472474
0.03413256166844314
Kappa:
0.7828141661597642
0.05687191252163874


In [40]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = logreg1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Logistic Regression")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

Logistic Regression
Accuracy:
0.8926470588235296
0.02473912329597165
Precision:
0.8940272921083269
0.022851773261301687
Recall:
0.9255659047999147
0.03267390367067739
Kappa:
0.7772979174941406
0.05125036340985312


In [41]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = randFor1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Random Forest")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

Random Forest
Accuracy:
0.8867647058823529
0.023885178042120724
Precision:
0.9071027628914925
0.030609493712952525
Recall:
0.9056873882663352
0.029737409469969455
Kappa:
0.7631234146885342
0.04960913138974293


In [42]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = naivebayes1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Naive Bayes")
print(np.mean(acc))
print(np.std(acc))
print(np.mean(prec))
print(np.std(prec))
print(np.mean(rec))
print(np.std(rec))
print(np.mean(kap))
print(np.std(kap))
#svm1, logreg1, randFor1, KNN1, neural_net

Naive Bayes
0.848529411764706
0.03010219042714517
0.9410419653841517
0.022948356835321267
0.8304706793149128
0.03589490473335775
0.6719690209178171
0.0647844456574809


In [43]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = KNN1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("k Nearest Neighbors")
print(np.mean(acc))
print(np.std(acc))
print(np.mean(prec))
print(np.std(prec))
print(np.mean(rec))
print(np.std(rec))
print(np.mean(kap))
print(np.std(kap))
#svm1, logreg1, randFor1, KNN1, neural_net

k Nearest Neighbors
0.7411764705882353
0.03568411973671318
0.7300384983699824
0.062329624158244036
0.8202823431108742
0.03227315195616186
0.4733597257464508
0.06946362018988508
